In [1]:
import csv
import datetime
import json

import ccxt
import pandas as pd
import pandas_ta as ta # needed for processing chart data 
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [2]:
news = {}
news_file="full_news_labeled.csv"
with open(news_file) as csvfile:
    reader = csv.reader(csvfile)
    for i, row in tqdm(enumerate(reader)):
        if i == 0:
            columns = row
            print(columns)
            continue
        date = f"{row[3]}-{row[4]}-{row[5]}-{row[6]}"
        text = f"{row[0]}. {row[1]}"
        news[date] = text
k = list(news.keys())
print(k[0], k[-1])

29838it [00:00, 107589.80it/s]

['title', 'content', 'labels', 'year', 'month', 'day', 'hour']
2018-2-1-14 2022-4-15-23


In [2]:
tokenizer = AutoTokenizer.from_pretrained("totoro4007/cryptodeberta-base-all-finetuned")

model = AutoModelForSequenceClassification.from_pretrained("totoro4007/cryptodeberta-base-all-finetuned")

_classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=1)

classifier = lambda x: _classifier(x, top_k=3)

sample = classifier("it is an example!")
print(sample)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'label': 'LABEL_2', 'score': 0.5892046093940735}, {'label': 'LABEL_0', 'score': 0.37871405482292175}, {'label': 'LABEL_1', 'score': 0.032081350684165955}]


In [3]:
def res2vec(res):
    vec = [0, 0, 0]
    for r in res:
        label = int(r['label'].split('_')[-1])
        score = round(r['score'], 4)
        vec[label] = score
    return vec

def vec_sum(vec1, vec2):
    return [vec1[0]+vec2[0], vec1[1]+vec2[1], vec1[2]+vec2[2]]

In [4]:
news = {}
news_file="full_news_labeled.csv"
with open(news_file) as csvfile:
    reader = csv.reader(csvfile)
    for i, row in tqdm(enumerate(reader)):
        if i == 0:
            columns = row
            print(columns)
            continue
        date = f"{row[3]}-{row[4]}-{row[5]}-{row[6]}"
        text = f"{row[0]}. {row[1]}"
        
        res = classifier(text)
        res_vec = res2vec(res)
        
        mask = [0, 0, 0]
        mask[int(row[2])] = 1
        
        if date in news:
            news[date][0] += 1
            news[date][1] = vec_sum(news[date][1], res_vec)
            news[date][2] = vec_sum(news[date][2], mask)
        else:
            news[date] = [1, res_vec, mask]
        
        #print(date, news[date])
        #break

['title', 'content', 'labels', 'year', 'month', 'day', 'hour']


/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [29]:
def dates2datetimes(dates):
    res = []
    for date in dates:
        date = list(map(int, date.split('-')))
        neo = datetime.datetime(date[0], date[1], date[2], date[3])
        res.append(neo)
    return res

def closest(query, dates):
    dates = dates2datetimes(dates)
    st = 0
    en = len(dates)-1
    while True:
        mid = (st + en) // 2
        if dates[mid] < query:
            st = mid
        else:
            en = mid
        if mid == (st + en) // 2:
            break            
    return mid

def isin(query, date):
    date = list(map(int, date.split('-')))
    date = datetime.datetime(date[0], date[1], date[2], date[3])
    hour = datetime.timedelta(hours=1)
    delta = query - date
    delta = delta / hour
    if 0 < delta and delta <= 4:
        return True
    return False

In [62]:
with open("BTC_USDT-4h_interval.json") as f: 
    d = json.load(f) 
    
chart_df = pd.DataFrame(d) 
chart_df = chart_df.rename(columns={0:"timestamp",
                                    1:"open",
                                    2:"high",
                                    3:"low", 
                                    4:"close",
                                    5:"volume"}) 

def process(df):
    binance = ccxt.binance() 
    dates = df['timestamp'].values 
    timestamp = [] 
    for i in range(len(dates)):
        date_string = binance.iso8601(int(dates[i])) 
        date_string = date_string[:10] + " " + date_string[11:-5] 
        timestamp.append(date_string) 
    df['datetime'] = timestamp 
    df = df.drop(columns={'timestamp'}) 
    return df

chart_df = process(chart_df) 

hours, days, months, years = [],[],[],[] 

for dt in tqdm(chart_df['datetime']):
    dtobj = pd.to_datetime(dt) 
    hour = dtobj.hour 
    day = dtobj.day 
    month = dtobj.month 
    year = dtobj.year 
    hours.append(hour)
    days.append(day) 
    months.append(month)
    years.append(year) 
    
chart_df['years'], chart_df['months'], chart_df['days'], chart_df['hours'] = years, months, days, hours
    
high_change, low_change = [], [] 
close = chart_df['close'].values 
high = chart_df['high'].values 
low = chart_df['low'].values 
for i in range(close.shape[0]-1):
    high_delta = (high[i+1] - close[i]) / close[i] 
    low_delta = (low[i+1] - close[i]) / close[i]
    high_change.append(high_delta) 
    low_change.append(low_delta)
high_change.append(None) 
low_change.append(None)

chart_df['high_delta'] = high_change
chart_df['low_delta'] = low_change 

chart_df['sent_0'] = 0.0
chart_df['sent_1'] = 0.0
chart_df['sent_2'] = 0.0

chart_df.dropna(inplace=True) 

chart_df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10560/10560 [00:00<00:00, 11886.87it/s]


,open,high,low,close,volume,datetime,years,months,days,hours,high_delta,low_delta,sent_0,sent_1,sent_2
0,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 04:00:00,2017,8,17,4,0.031127,-0.003832,0.0,0.0,0.0
1,4333.32,4485.39,4333.32,4427.30,63.619882,2017-08-17 08:00:00,2017,8,17,8,0.013121,-0.021205,0.0,0.0,0.0
2,4436.06,4485.39,4333.42,4352.34,174.562001,2017-08-17 12:00:00,2017,8,17,12,0.000574,-0.034832,0.0,0.0,0.0
3,4352.33,4354.84,4200.74,4325.23,225.109716,2017-08-17 16:00:00,2017,8,17,16,0.010279,-0.015414,0.0,0.0,0.0
4,4307.56,4369.69,4258.56,4285.08,249.769913,2017-08-17 20:00:00,2017,8,17,20,0.012961,-0.035115,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10554,22485.27,22796.71,20846.00,22127.91,64139.981750,2022-06-14 00:00:00,2022,6,14,0,0.055811,-0.016459,0.0,0.0,0.0
10555,22127.92,23362.88,21763.71,22877.52,43892.535610,2022-06-14 04:00:00,2022,6,14,4,0.004478,-0.050368,0.0,0.0,0.0
10556,22877.52,22979.96,21725.22,21994.56,22494.024070,2022-06-14 08:00:00,2022,6,14,8,0.036620,-0.008345,0.0,0.0,0.0
10557,21994.56,22800.00,21811.01,22680.38,29165.257260,2022-06-14 12:00:00,2022,6,14,12,0.004263,-0.024426,0.0,0.0,0.0


In [63]:
for index, row in tqdm(chart_df.iterrows()):
    query = datetime.datetime(row['years'], row['months'], row['days'], row['hours'])
    lk = list(news.keys())
    idx = closest(query, lk)
    ks = []
    n = 0
    for k in lk[max(0, idx-5):idx]:
        if isin(query, k):
            ks.append(k)
            n += news[k][0]
    for k in ks:
        v = news[k]
        chart_df.at[index, 'sent_0'] += v[1][0] / n
        chart_df.at[index, 'sent_1'] += v[1][1] / n
        chart_df.at[index, 'sent_2'] += v[1][2] / n
        
chart_df

10559it [03:36, 48.67it/s]


,open,high,low,close,volume,datetime,years,months,days,hours,high_delta,low_delta,sent_0,sent_1,sent_2
0,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 04:00:00,2017,8,17,4,0.031127,-0.003832,0.0,0.0,0.0
1,4333.32,4485.39,4333.32,4427.30,63.619882,2017-08-17 08:00:00,2017,8,17,8,0.013121,-0.021205,0.0,0.0,0.0
2,4436.06,4485.39,4333.42,4352.34,174.562001,2017-08-17 12:00:00,2017,8,17,12,0.000574,-0.034832,0.0,0.0,0.0
3,4352.33,4354.84,4200.74,4325.23,225.109716,2017-08-17 16:00:00,2017,8,17,16,0.010279,-0.015414,0.0,0.0,0.0
4,4307.56,4369.69,4258.56,4285.08,249.769913,2017-08-17 20:00:00,2017,8,17,20,0.012961,-0.035115,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10554,22485.27,22796.71,20846.00,22127.91,64139.981750,2022-06-14 00:00:00,2022,6,14,0,0.055811,-0.016459,0.0,0.0,0.0
10555,22127.92,23362.88,21763.71,22877.52,43892.535610,2022-06-14 04:00:00,2022,6,14,4,0.004478,-0.050368,0.0,0.0,0.0
10556,22877.52,22979.96,21725.22,21994.56,22494.024070,2022-06-14 08:00:00,2022,6,14,8,0.036620,-0.008345,0.0,0.0,0.0
10557,21994.56,22800.00,21811.01,22680.38,29165.257260,2022-06-14 12:00:00,2022,6,14,12,0.004263,-0.024426,0.0,0.0,0.0


In [64]:
#print(chart_df.iloc[10199])
for index, row in tqdm(chart_df.iterrows()):
    if row.sent_0 > 0:
        print(row)

3567it [00:00, 17780.60it/s]

open                     11171.71
high                      11250.0
low                       10881.0
close                    11126.06
volume                7744.962027
datetime      2018-02-19 16:00:00
years                        2018
months                          2
days                           19
hours                          16
high_delta               0.007724
low_delta               -0.010881
sent_0                     0.1198
sent_1                     0.5601
sent_2                       0.32
Name: 1111, dtype: object
open                      10855.0
high                      10950.0
low                       10780.0
close                     10937.0
volume                2406.192253
datetime      2018-03-02 16:00:00
years                        2018
months                          3
days                            2
hours                          16
high_delta               0.017647
low_delta               -0.004223
sent_0                     0.1587
sent_1                

5219it [00:00, 5910.83it/s] 


open                     11421.93
high                     11444.42
low                      11017.59
close                    11257.82
volume               13544.435039
datetime      2019-07-03 16:00:00
years                        2019
months                          7
days                            3
hours                          16
high_delta               0.065205
low_delta               -0.004692
sent_0                   0.318433
sent_1                     0.4739
sent_2                   0.207667
Name: 4098, dtype: object
open                     11256.97
high                     11991.89
low                       11205.0
close                     11940.0
volume               18740.457371
datetime      2019-07-03 20:00:00
years                        2019
months                          7
days                            3
hours                          20
high_delta               0.005025
low_delta               -0.019172
sent_0                     0.2545
sent_1               

5849it [00:00, 4104.54it/s]

...


open                      40731.0
high                      41120.0
low                      40387.77
close                    40507.56
volume                9492.764065
datetime      2021-08-06 08:00:00
years                        2021
months                          8
days                            6
hours                           8
high_delta               0.048094
low_delta               -0.005124
sent_0                     0.2178
sent_1                     0.3732
sent_2                      0.409
Name: 8684, dtype: object
open                     40507.56
high                     42455.73
low                       40300.0
close                    42199.98
volume               16989.759862
datetime      2021-08-06 12:00:00
years                        2021
months                          8
days                            6
hours                          12
high_delta               0.028257
low_delta                    -0.0
sent_0                     0.1588
sent_1               

9771it [00:02, 2120.27it/s]

open                     56318.89
high                     57045.44
low                      56229.98
close                    56733.69
volume                 5200.45315
datetime      2021-12-03 04:00:00
years                        2021
months                         12
days                            3
hours                           4
high_delta               0.010511
low_delta               -0.005417
sent_0                   0.144243
sent_1                   0.315371
sent_2                   0.540405
Name: 9397, dtype: object
open                      56733.7
high                      57330.0
low                      56426.35
close                    56959.77
volume                 5749.03202
datetime      2021-12-03 08:00:00
years                        2021
months                         12
days                            3
hours                           8
high_delta                0.01124
low_delta               -0.021731
sent_0                    0.15167
sent_1                

10559it [00:02, 3694.63it/s]


open                     38729.57
high                     39327.99
low                      38660.52
close                    39088.94
volume                 4198.07907
datetime      2022-03-12 00:00:00
years                        2022
months                          3
days                           12
hours                           0
high_delta               0.004414
low_delta                -0.00402
sent_0                    0.21749
sent_1                    0.23367
sent_2                    0.54881
Name: 9990, dtype: object
open                     39088.93
high                     39261.46
low                       38931.8
close                    39104.99
volume                 2950.84658
datetime      2022-03-12 04:00:00
years                        2022
months                          3
days                           12
hours                           4
high_delta               0.004589
low_delta               -0.003772
sent_0                     0.2145
sent_1               

In [65]:
chart_df.set_index(pd.DatetimeIndex(chart_df['datetime']), inplace=True) 

chart_df['bop'] = chart_df.ta.bop(lookahead=False)
chart_df['ebsw'] = chart_df.ta.ebsw(lookahead=False) 
chart_df['cmf'] = chart_df.ta.cmf(lookahead=False) 
chart_df['rsi/100'] = chart_df.ta.rsi(lookahead=False) / 100 
chart_df['vwap'] = chart_df.ta.vwap(lookahead=False) 
chart_df['high/low'] = chart_df['high'] / chart_df['low'] 
chart_df['close/open'] = chart_df['close'] / chart_df['open'] 
chart_df['high/open'] = chart_df['high'] / chart_df['open'] 
chart_df['low/open'] = chart_df['low'] / chart_df['open'] 

chart_df['hwma'] = chart_df.ta.hwma(lookahead=False)
chart_df['linreg'] = chart_df.ta.linreg(lookahead=False)
chart_df['hwma/close'] = chart_df['hwma'] / chart_df['close'] 
chart_df['linreg/close'] = chart_df['linreg'] / chart_df['close']

for i in tqdm(range(1, 4)): 
    for col in ['open', 'high', 'low', 'close', 'volume', 'vwap']:
        val = chart_df[col].values 
        val_ret = [None for _ in range(i)] 
        for j in range(i, len(val)): 
            if val[j-i] == 0:
                ret = 1 
            else:
                ret = val[j] / val[j-i] 
            val_ret.append(ret) 
        chart_df['{}_change_{}'.format(col, i)] = val_ret 
            
    

chart_df.dropna(inplace=True) 
chart_df.drop(columns={'datetime', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'hwma', 'linreg', 'years'}, inplace=True) 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 12.45it/s]


In [66]:
chart_df.head(2)

,months,days,hours,high_delta,low_delta,sent_0,sent_1,sent_2,bop,ebsw,...,low_change_2,close_change_2,volume_change_2,vwap_change_2,open_change_3,high_change_3,low_change_3,close_change_3,volume_change_3,vwap_change_3
datetime,,,,,,,,,,,,,,,,,,,,,
2017-08-23 16:00:00,8,23,16,0.014957,-0.011481,0.0,0.0,0.0,-0.703979,0.000000,...,1.011464,0.971468,1.293382,1.009008,1.034863,1.034916,1.016898,1.010354,1.231233,1.016728
2017-08-23 20:00:00,8,23,20,0.022008,-0.007049,0.0,0.0,0.0,-0.206431,0.577350,...,0.980521,0.980215,0.642973,0.999909,1.013438,0.984992,1.003155,0.970746,1.130291,1.008057
2017-08-24 00:00:00,8,24,0,0.015573,-0.005637,0.0,0.0,0.0,-0.279572,0.797082,...,0.995492,0.999152,0.707294,0.997091,0.985977,0.985642,0.984186,0.980112,0.520390,0.997940
2017-08-24 04:00:00,8,24,4,0.023504,-0.000799,0.0,0.0,0.0,0.455473,0.983912,...,1.005059,1.009555,0.807118,0.998763,0.973182,0.980763,0.996803,1.008805,0.705343,0.997822


In [67]:
chart_df.to_csv('deberta.csv')